In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 7 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line
Delta_x = 1
Delta_conv = 1
y_dist = 3
thick_conv = 0.005
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

x_start_up, y_start_up = x_vals[index_line_bump - Delta_conv], (y_dist * max(y_vals_top)) + thick_conv
angle_deg = -5
angle_rad = np.radians(angle_deg)

# Define the length of the line segment
End_line_param = x_vals[-1] + 4 - x_vals[-1]
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
y_end_up = y_start_up

# Define Far Field
Y_DIST = 250
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -25
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = 25 * x_vals[np.argmax(x_vals)]

# .geo file creation:
with open("geometry.geo", "w") as file:

    file.write("h = 1.0;\n") # Size of cells close to bump and convergent
    file.write("H = 1.0;\n") # Size of cells close to farfield

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
        file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")
    for i in range(len(x_vals) - 1):
        file.write(f"Line({i + 1}) = {{{i + 1}, {i + 2}}};\n")

    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1

    # Write the convex shape points and lines
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h}};\n")
    file.write(f"Line({len(x_vals) + 1}) = {{{new_point_start}, {new_point_end}}};\n") # line 8: 8-> 9

    file.write(f"Point({upper_point_start}) = {{{x_start_up}, {y_start_up}, 0, h}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h}};\n")
    file.write(f"Line({len(x_vals) + 2}) = {{{upper_point_start}, {upper_point_end}}};\n") # line 9: 10->11

    # Define connecting lines for closed convex surface
    file.write(f"Line({len(x_vals) + 3}) = {{{new_point_start}, {upper_point_start}}};\n") # Wannabe smusso line 10 8->10
    file.write(f"Line({len(x_vals) + 4}) = {{{new_point_end}, {upper_point_end}}};\n") # Vertical conncection line 11 9->11

    # Boundary points for the entire domain
    boundary_point_1 = upper_point_end + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, H}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, 0, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_5}) = {{{X_START}, 0, 0, H}};\n")

    # Define lines for the main boundary
   # file.write(f"Line({len(x_vals) + 5}) = {{{new_point_start - 1}, {1}}};\n")  # Connect end bump to start bump
    file.write(f"Line({len(x_vals) + 6}) = {{{boundary_point_1}, {boundary_point_2}}};\n")
    file.write(f"Line({len(x_vals) + 7}) = {{{boundary_point_2}, {boundary_point_3}}};\n")
    file.write(f"Line({len(x_vals) + 8}) = {{{boundary_point_3}, {boundary_point_4}")
    file.write("};\n")
    file.write(f"Line({len(x_vals) + 9}) = {{{boundary_point_4}, {boundary_point_5}}};\n")
    file.write(f"Line({len(x_vals) + 10}) = {{{boundary_point_5}, 1}};\n")  # Connect back to start of bump
    file.write(f"Line({len(x_vals) + 11}) = {{{new_point_start - 1},{boundary_point_1} }};\n")

    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    file.write("Line Loop(1) = {")
    file.write(", ".join(f"{i + 1}" for i in range(len(x_vals)-1)))  # Lines on the bump boundary
    #file.write(f",{len(x_vals) + 5}, {len(x_vals) + 11}, {len(x_vals) + 6}, {len(x_vals) + 7}, {len(x_vals) + 8}, {len(x_vals) + 9}, {len(x_vals) + 10}")
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 6}, {len(x_vals) + 7}, {len(x_vals) + 8}, {len(x_vals) + 9}, {len(x_vals) + 10}")
    #file.write("};\nPlane Surface(1) = {1};\n")
    file.write("};\n")

    file.write("Line Loop(2) = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2}, -{len(x_vals) + 3}")
    #file.write("};\nPlane Surface(2) = {2};\n")
    file.write("};\n")

    file.write("Plane Surface(1) = {1, 2};\n") # It tells GMSH to mesh only on 1 and not account for 2

    # Physical Surface definition
    file.write("Physical Surface('VOLUME') = {1};\n")

    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{len(x_vals) + 9}}};\n")

    file.write("Physical Line('lower wall') = {")
    file.write(f"{len(x_vals) + 10},")
    file.write(", ".join(f"{i + 1}" for i in range(len(x_vals)-1)))
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 6}")
    file.write("};\n")

    file.write(f"Physical Line('upper wall') = {{{len(x_vals) + 8}}};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2}, -{len(x_vals) + 3}")
    file.write("};\n")

    file.write(f"Physical Line('outlet') = {{{len(x_vals) + 7}}};\n")


    # MESH:

    file.write("Mesh.Algorithm = 6; // Frontal-Delaunay for quads\n")  # Try algorithm 6 for quads
    file.write("Mesh.RecombineAll = 1; // Enable recombination for quad elements\n")

    # Structured mesh on the bump
    file.write(f"Transfinite Curve {{")
    file.write(", ".join(f"{i + 1}" for i in range(len(x_vals) - 1)))  # Lines forming the bump
    file.write(f"}} = {N_points_bump} Using Progression 1.0;\n")  # Number of points and progression type



# Download the file (if running in an environment that supports file download)
from google.colab import files
files.download("geometry.geo")